In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import datetime
from bson import ObjectId
from pymongo import MongoClient
from scipy.optimize import minimize

In [2]:
client = MongoClient('127.0.0.1', 27017)
db = client.moniter_crowdfunding
# db.authenticate(name="craw", password="craw")

In [3]:
project = db.projects
s_project = db.sucess_projects
f_project = db.failure_projects
front_page = db.front_page

In [5]:
s_project.find_one()['_id']

'100010'

In [10]:
import sys

In [11]:
def iterd(n):
    try:
        for i in range(n):
            yield i
    except StopIteration:
        sys.exit()

In [ ]:
class IterProj(object):
    def __init__(self, pids, projects, front_page):
        self.index = 0
        self.pids = pids
        self.proj = projects.find_one({'_id': self.pids[self.index]})
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.index < len(self.pids):
            proj = ProjData()
        
        

In [13]:
a = iterd(10)
for i in range(10):
    print(next(a))

0
1
2
3
4
5
6
7
8
9


# 处理评论数据

In [ ]:
proj = s_project.find_one({"项目动态信息.0.爬取时间": {'$gte': datetime.datetime(2019, 1, 1), 
                                       '$lte': datetime.datetime(2019, 1, 30)}})

In [ ]:
proj.keys()

In [ ]:
review = proj['评论']

In [ ]:
review.keys()

In [ ]:
review['爬取时间'], review['总页数'], review['总评论数']

In [ ]:
review_spe = review['评论详细']

In [ ]:
review_spe.keys()

In [ ]:
keys = ['createTime', 'topicId', 'nicknameShow', 'topicContent', 'likeCount', 'replys']

In [ ]:
for t_id in review_spe:
    print(t_id)
    for key in keys:
        print('    ', key, review_spe[t_id][key])
    print('\n')

In [ ]:
class ProjData:
    def __init__(self, proj, diffu_col_names, review_col_names, front_page):
        self.proj = proj
        self.front_page = front_page
        self.diffu_col_names = diffu_col_names
        self.review_col_names = review_col_names
        try:
            self.id = self.proj["_id"]
            self.name = self.proj["项目名称"]
            self.state = self.proj["状态"]
            self.duration = self.proj["众筹期限"]
            self.aim_fund = self.proj["目标金额"]
            self.final_fund = self.proj["项目动态信息"][-1]["筹集金额"]
            self.company_name = self.proj["公司名称"]
            self.company_phone = self.proj["公司电话"]
            self.link = self.proj["发起人链接"]
            self.category = self.proj["所属类别"]
            self.review = self.proj['评论']
            self.num_review = self.proj['评论']['总评论数']
            
            self.review_df = self.get_review_data()

#             self.pure_diffu_df = self.get_pure_diffu()  # 未处理的扩散数据
#             self.front_df = self.get_front_position()  # 首页数据
#             self.diffu_df = self.merge_diffu_position()
#             self.diffu_day_df = self.to_diffu_day_df()  # 每天最晚时间点的扩散数据
#             self.cum_diffu_df = self.prepare_data()  # 索引为相对众筹开始时间点的间隔
        except Exception as e:
            print(e)
    
    def get_review_data(self):
        raw_review = self.review['评论详细']
        indices = []
        values = []
        for t_id in raw_review:
            value = []
            for key in self.review_col_names:
                if key == 'createTime':
                    indices.append(raw_review[t_id][key])
                else:
                    value.append(raw_review[t_id][key])
                
            values.append(value)
        columns = self.review_col_names[1:]
        return pd.DataFrame(values, index=indices, columns=columns)
            

In [ ]:
diffu_col_names = ["星期", "关注数", "支持者数", "点赞数", "完成百分比", "筹集金额"]
review_col_names = ['createTime', 'topicId', 'nicknameShow', 'topicContent', 'likeCount', 'replys']
proj = s_project.find_one({'项目动态信息.0.爬取时间': {'$gte': datetime.datetime(2019, 1, 1),
                                         '$lte': datetime.datetime(2019, 1, 10)}})

In [ ]:
p = ProjData(proj, diffu_col_names, review_col_names, front_page)

In [ ]:
p.review_df